In [1]:
import os
import numpy as np
import pandas as pd
from numba import jit
from tqdm import tqdm
import statsmodels.api as sm
import sys
sys.path.append('D:\实习\研报复现\QuantDev\ymtool')
import Alpha as alpha

import warnings
warnings.filterwarnings('ignore')

# CDPP

In [2]:
def cdpp_func(date):
    close_path = os.path.join(r"D:\实习\研报复现\dataAll\stockMinData\close",date)
    date = pd.to_datetime(date.split('.')[0],format='%Y%m%d')
    close = pd.read_parquet(close_path)
    code = close.columns
    close = close.values
    
    dp = np.diff(close,axis=0)[:-1]
    
    # CDPP_pos
    dp_pos = np.where(dp>0,dp,np.nan)
    p_pos = np.where(dp>0,close[2:],np.nan)
    cov = np.nanmean((p_pos-np.nanmean(p_pos,axis=0))*(dp_pos-np.nanmean(dp_pos,axis=0)),axis=0)
    cdpp_pos = cov/np.nanstd(p_pos,axis=0)/np.nanstd(dp_pos,axis=0)

    # CDPP_neg
    dp_neg = np.where(dp<0,dp,np.nan)
    p_neg = np.where(dp<0,close[2:],np.nan)
    cov = np.nanmean((p_neg-np.nanmean(p_neg,axis=0))*(dp_neg-np.nanmean(dp_neg,axis=0)),axis=0)
    cdpp_neg = cov/np.nanstd(p_neg,axis=0)/np.nanstd(dp_neg,axis=0)
    
    cdpp_pos = pd.Series(cdpp_pos,index=code, name=date)
    cdpp_neg = pd.Series(cdpp_neg,index=code, name=date)
    
    return cdpp_pos,cdpp_neg
    
data = os.listdir(r"D:\实习\研报复现\dataAll\stockMinData\close")

CDPP_POS = []
CDPP_NEG = []

for i in tqdm(data):
    
    cdpp_pos,cdpp_neg = cdpp_func(date=i)
    CDPP_POS.append(cdpp_pos)
    CDPP_NEG.append(cdpp_neg)
    
CDPP_POS = pd.concat(CDPP_POS, axis=1).T
CDPP_NEG = pd.concat(CDPP_NEG, axis=1).T

100%|██████████████████████████████████████████████████████████████████████████████| 2512/2512 [13:40<00:00,  3.06it/s]


In [7]:
dP_pos_P_Corr = alpha.mvind_neutral(CDPP_POS.rolling(20,min_periods=1).mean().dropna(axis=0,how='all').iloc[19:].loc[:'2024-03'])
dP_neg_P_Corr =alpha.mvind_neutral(CDPP_NEG.rolling(20,min_periods=1).mean().dropna(axis=0,how='all').iloc[19:].loc[:'2024-03'])

CDPP = alpha.zscore(dP_pos_P_Corr)-alpha.zscore(dP_neg_P_Corr)

100%|██████████████████████████████████████████████████████████████████████████████| 2473/2473 [00:40<00:00, 60.76it/s]


In [9]:
alpha.to_fctDataBase(CDPP.dropna(axis=0,how='all'),'CDPP')

# CDCDP

In [3]:
def cdpdp_func(date):
    close_path = os.path.join(r"D:\实习\研报复现\dataAll\stockMinData\close",date)
    date = pd.to_datetime(date.split('.')[0],format='%Y%m%d')
    close = pd.read_parquet(close_path)
    code = close.columns
    close = close.values
    
    dp_t = np.diff(close,axis=0)[:-1]
    dp_t_1 = np.diff(close,axis=0)[1:]
    # dpdp_pos
    dp_t_pos = np.where(np.logical_and(dp_t > 0, dp_t_1 > 0), dp_t, np.nan)
    dp_t_1_pos = np.where(np.logical_and(dp_t > 0, dp_t_1 > 0), dp_t_1, np.nan)
    cov = np.nanmean((dp_t_pos-np.nanmean(dp_t_pos,axis=0))*(dp_t_1_pos-np.nanmean(dp_t_1_pos,axis=0)),axis=0)
    dpdp_pos = cov/np.nanstd(dp_t_pos,axis=0)/np.nanstd(dp_t_1_pos,axis=0)

    # dpdp_neg
    dp_t_neg = np.where(np.logical_and(dp_t < 0, dp_t_1 < 0), dp_t, np.nan)
    dp_t_1_neg = np.where(np.logical_and(dp_t < 0, dp_t_1 < 0), dp_t_1, np.nan)
    cov = np.nanmean((dp_t_neg-np.nanmean(dp_t_neg,axis=0))*(dp_t_1_neg-np.nanmean(dp_t_1_neg,axis=0)),axis=0)
    dpdp_neg = cov/np.nanstd(dp_t_neg,axis=0)/np.nanstd(dp_t_1_neg,axis=0)
    
    dpdp_pos = pd.Series(dpdp_pos,index=code, name=date)
    dpdp_neg = pd.Series(dpdp_neg,index=code, name=date)
    
    return dpdp_pos,dpdp_neg
    
data = os.listdir(r"D:\实习\研报复现\dataAll\stockMinData\close")

DPDP_POS = []
DPDP_NEG = []

for i in tqdm(data):
    
    dpdp_pos,dpdp_neg = cdpdp_func(date=i)
    DPDP_POS.append(dpdp_pos)
    DPDP_NEG.append(dpdp_neg)
    
DPDP_POS = pd.concat(DPDP_POS, axis=1).T
DPDP_NEG = pd.concat(DPDP_NEG, axis=1).T

100%|██████████████████████████████████████████████████████████████████████████████| 2512/2512 [12:13<00:00,  3.43it/s]


In [ ]:
dp_pos_dp_pos_corr = alpha.mvind_neutral(DPDP_POS.rolling(20,min_periods=1).mean().dropna(axis=0,how='all').iloc[19:].loc[:'2024-03'])
dp_neg_dp_neg_corr =alpha.mvind_neutral(DPDP_NEG.rolling(20,min_periods=1).mean().dropna(axis=0,how='all').iloc[19:].loc[:'2024-03'])

CDPDP = alpha.zscore(dp_pos_dp_pos_corr)+alpha.zscore(dp_neg_dp_neg_corr)

alpha.to_fctDataBase(CDPDP.dropna(axis=0,how='all'),'CDPDP')